In [8]:
# ruff: noqa: F401, E402
import math
import sys
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    starmap,
    tee,
)
from math import comb, factorial, log, sqrt
from operator import add, and_, contains, indexOf, itemgetter, neg, or_, xor
from pprint import pprint
from typing import Callable, Iterable, List, Tuple

import matplotlib.pyplot as plt
import numpy as np

import arrays
import graphs
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import strings
import trees

for m in (
    arrays,
    graphs,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    strings,
    trees,
):
    reload(m)


from parsing import format_binop_expression, parse_binop_expression
from sequences import find_if
from strings import StringView

print()


In [253]:
def comod(n: int, k: int, mod: int) -> int:
    "Return binominal, combinatorial factor modulo `mod`"
    if 0 >= k or k >= n:
        return int(0 <= k <= n)
    k = min(k, n - k)
    g = lambda a, x: a * x % mod
    fk = reduce(g, range(1, k + 1), 1)
    fn = reduce(g, range(n - k + 1, n + 1), 1)
    return fn * pow(fk, -1, mod) % mod


In [245]:
comb(1, 1)


1

In [247]:
comb(6, 2)


15

In [248]:
mod = 1000_000_007
comod(6, 2, mod)


15

In [239]:
10 * pow(10, mod - 2, mod) % mod


1

In [254]:
def count_grid_paths(n: int, m: int) -> int:
    "Count the number of paths form (n, m) -> (0, 0)."
    return comod(m+n, n, 1000000007)


In [255]:
count_grid_paths(3, 0)


1

In [256]:
count_grid_paths(3, 6)


84

In [257]:
count_grid_paths(35, 31)


21144547

In [258]:
count_grid_paths(6, 6)


924

In [ ]:
6 6
